In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid           # Downloads standard citation network datasets (Cora, Citeseer, Pubmed)
from torch_geometric.transforms import NormalizeFeatures # Normalizes node features to sum to 1 (helps training)
from torch_geometric.nn import SGConv
from sklearn.metrics import accuracy_score

In [2]:
# 1. Download and preprocess data

# Device Configuration - Purpose: Use GPU if available for faster training.
"""
   torch.cuda.is_available(): Checks if CUDA-enabled GPU is available
   device variable: Will be either 'cuda' (GPU) or 'cpu' (CPU)
"""

device  = 'cuda' if torch.cuda.is_available() else 'cpu'
    
# Load Cora dataset with normalized features
"""
     root: Directory where dataset will be stored/downloaded
     name='Cora': Specifies the Cora dataset (citation network)
     transform=NormalizeFeatures(): Applies feature normalization

    dataset[0]: Accesses the first (and only) graph in the dataset
    .to(device): Moves all tensors (features, edges, etc.) to GPU/CPU
"""
dataset = Planetoid(root='C:/Users/Majid/Downloads/CNN', name='Cora', transform=NormalizeFeatures())
data = dataset[0].to(device)  # Move graph data to GPU/CPU
    
# Print dataset info
print(f"Dataset: {dataset}")
print(f"First 5 nodes features : {data.x[:5]}\n")
print(f"First 5 edges: {data.edge_index[:5]}\n")
print(f"First 5 labels: {data.y[:5]}\n")
print(f"First 5 test mask: {data.test_mask[:5]}")
print(f"First 5 train mask: {data.train_mask[:5]}")
print(f"First 5 validation mask: {data.val_mask[:5]}")
   
print(f"Number of nodes: {data.num_nodes}")
print(f"Number of edges: {data.num_edges}")
print(f"Number of features: {dataset.num_features}")
print(f"Number of classes: {dataset.num_classes}")

"""
    data: The complete graph object containing:
       - Node features (data.x)
       - Edge indices (data.edge_index)
       - Labels (data.y)
       - Train/val/test masks (data.train_mask, etc.)
    dataset.num_features: Number of input features per node (1,433)
    dataset.num_classes: Number of output classes (7)
 """

Dataset: Cora()
First 5 nodes features : tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

First 5 edges: tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])

First 5 labels: tensor([3, 4, 4, 0, 3])

First 5 test mask: tensor([False, False, False, False, False])
First 5 train mask: tensor([True, True, True, True, True])
First 5 validation mask: tensor([False, False, False, False, False])
Number of nodes: 2708
Number of edges: 10556
Number of features: 1433
Number of classes: 7


'\n    data: The complete graph object containing:\n       - Node features (data.x)\n       - Edge indices (data.edge_index)\n       - Labels (data.y)\n       - Train/val/test masks (data.train_mask, etc.)\n    dataset.num_features: Number of input features per node (1,433)\n    dataset.num_classes: Number of output classes (7)\n '

In [3]:
# 2. Build Model

"""
    * Class Definition:
        - Inherits from nn.Module (base class for all PyTorch neural networks)
        - This inheritance provides essential functionality for model training and saving
        
    * __init__ Method (Initialization):
        - num_features: Dimension of input features per node (1433 for Cora)
        - num_classes: Number of output classes (7 for Cora)
        - K=2: Number of hops/propagation steps (default=2)
        - super().__init__(): Initializes the parent nn.Module class
        - self.conv: Creates the SGC layer using PyG's SGConv

    * SGConv Layer:
        - Performs two key operations:
            * Feature Propagation: Smooths features over K-hop neighborhoods
            * Linear Transformation: Applies learned weights W
        - Combines these into single efficient operation
        - No nonlinear activation between steps (unlike traditional GCNs)
"""


class SGCModel(nn.Module):
    def __init__(self, num_features, num_classes, K=2):
        super().__init__()
        self.conv = SGConv(num_features, num_classes, K=K)


    """
        forward Method:
            - Defines how data flows through the network
            - Parameters:
                * x: Node feature matrix (shape: [num_nodes, num_features])
                * edge_index: Graph connectivity (shape: [2, num_edges])
                
            - Simply passes these through the SGC layer
            - Output: logits for each node (shape: [num_nodes, num_classes])

            - Key Characteristics of This Model: 
                Only one learnable layer (the SGConv)
                No hidden layers or nonlinear activations

            K=2 means the model considers 2-hop neighborhoods
    """
    
    def forward(self, x, edge_index):
        return self.conv(x, edge_index)

In [4]:
# Initialize model

# Creates a new instance of our SGCModel class
model = SGCModel(
    num_features=dataset.num_features, # Sets the input dimension to match the dataset's feature size. For Cora dataset: 1433 
    num_classes=dataset.num_classes,   # Sets output dimension to number of target classes. For Cora labels: 7 
    K=2  # Number of propagation hops. K=2 means the model considers 2-hop neighborhoods
)

In [5]:
# 3. Fitting and embedding

"""
   optimizer: Adam optimizer with:
   Learning rate (lr=0.2) - Higher than typical (0.001) because SGC trains fast
   Weight decay (5e-4) - L2 regularization to prevent overfitting
   criterion: CrossEntropyLoss - Standard for multi-class classification
"""
optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

def train():
    # Sets model to training mode (important for dropout/batch norm if used)
    model.train()    

    # Resets gradients from previous iteration
    optimizer.zero_grad()   
    
    # Calls the forward method of SGCModel
    # Input: Node features (data.x) and graph structure (data.edge_index)
    # Output: Class predictions for all nodes
    out = model(data.x, data.edge_index)

    #  Computed only on training nodes (data.train_mask)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])

    # Computes gradients via backpropagation
    loss.backward()

    # Updates model parameters using gradients
    optimizer.step()
    
    return loss.item()

In [6]:
# 4. Prediction and evaluation
def test():

    """ Puts the model in evaluation mode
        Disables dropout and batch normalization layers if present
        Ensures consistent behavior during inference
    """
    model.eval()

    """ Runs the forward pass of the model
        Inputs: 
             data.x: Node feature matrix (shape: [num_nodes, num_features]) 
             data.edge_index: Graph connectivity (shape: [2, num_edges])
        data.edge_index: Graph connectivity (shape: [2, num_edges])
    """
    out = model(data.x, data.edge_index)

    """
       Converts logits to predicted class labels
       argmax(dim=1) selects the class with highest score for each node
       Result is a tensor of predicted class indices (shape: [num_nodes])
    """
    pred = out.argmax(dim=1)
    
    # Accuracy Calculation
    """
       Sums correct predictions (train_correct.sum())
       Divides by total nodes in split (train_mask.sum())
       Converts to Python int to avoid tensor types
       Results are floating-point accuracy values between 0 and 1
    """
    train_correct = pred[data.train_mask] == data.y[data.train_mask]
    val_correct = pred[data.val_mask] == data.y[data.val_mask]
    test_correct = pred[data.test_mask] == data.y[data.test_mask]
    
    train_acc = int(train_correct.sum()) / int(data.train_mask.sum())
    val_acc = int(val_correct.sum()) / int(data.val_mask.sum())
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
    
    return train_acc, val_acc, test_acc

In [7]:
# Training loop
# Runs for 100 complete passes through the dataset
for epoch in range(100):

    """
       Call train() function:
           Sets model to training mode (model.train())
           Performs forward pass to compute predictions
           Calculates loss on training nodes
           Calculates loss on training nodes
           Updates model parameters (optimizer.step())
           Updates model parameters (optimizer.step())
    """
    loss = train()

    """
        Calls test() function which:
            Sets model to evaluation mode (model.eval())
            Computes predictions on all data
            Computes predictions on all data
            
    """
    if epoch % 10 == 0:
        train_acc, val_acc, test_acc = test()
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 000, Loss: 1.9466, Train: 0.4286, Val: 0.3040, Test: 0.2850
Epoch: 010, Loss: 1.5314, Train: 0.9357, Val: 0.7780, Test: 0.8000
Epoch: 020, Loss: 1.5047, Train: 0.9214, Val: 0.7740, Test: 0.7860
Epoch: 030, Loss: 1.4927, Train: 0.9214, Val: 0.7580, Test: 0.7610
Epoch: 040, Loss: 1.4923, Train: 0.9143, Val: 0.7580, Test: 0.7710
Epoch: 050, Loss: 1.4962, Train: 0.9143, Val: 0.7720, Test: 0.7830
Epoch: 060, Loss: 1.4984, Train: 0.9143, Val: 0.7740, Test: 0.7770
Epoch: 070, Loss: 1.4986, Train: 0.9143, Val: 0.7740, Test: 0.7800
Epoch: 080, Loss: 1.4990, Train: 0.9143, Val: 0.7720, Test: 0.7800
Epoch: 090, Loss: 1.4986, Train: 0.9143, Val: 0.7720, Test: 0.7800


In [8]:
# Final evaluation
train_acc, val_acc, test_acc = test()
print(f'Final results: Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Final results: Train: 0.9143, Val: 0.7720, Test: 0.7800
